# BigQueryを学ぶ

In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

[]


In [82]:
from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "bitcoin_blockchain")
QUERY = "SELECT * FROM `bigquery-public-data.bitcoin_blockchain.blocks`"

In [73]:
df = bq_assistant.head("blocks",num_rows=100)

In [74]:
bq_assistant.

,block_id,previous_block,merkle_root,timestamp,difficultyTarget,nonce,version,work_terahash,work_error,transactions,row_number
0,0000000000000af5ee70ada24ec1c34c867ac73bcc7d9b...,0000000000001fffc5774fc8a02161ff1b20b73d79e59d...,f2886a527c015c67923262b3a550acd2cd45ef0b917d43...,1307330213000,438735905,3584907577,1,18678,None,[{'transaction_id': '09119c68cfc5c9d5926181ce5...,1
1,0000000000000000000b1525c527236512af52a89a5fce...,000000000000000000a01e0122b400234eb8e5cfdf2a35...,aad0ce06f914ba652480a6cd93937fb956dd978b41f66f...,1482015099000,402885509,1962274162,536870912,13321209935,None,[{'transaction_id': '38064838c2778a381bf9d95b0...,1
2,000000000000000038b34e3c3ebcba81be381f314161fe...,00000000000000000ceab750a0c3443479738b9b926ac8...,8a4655e47a5110b91aa5f7e9d20354c5e7ce7a99dbf1ac...,1407067666000,406498978,4226596330,2,804736316,None,[{'transaction_id': '86875d2d8c89449e2f7f65007...,1
3,00000000000003a7d5bbda9862fd58373bb547a160829e...,000000000000037e4a250ffb7900fb4bac68d5390db692...,7d05c93663d25800de976897b828b605e45acb67180892...,1352944336000,436533995,3238317857,2,144689,None,[{'transaction_id': '78b6f7e74d03b7b9da5ba3f5a...,1
4,000000000000021010be0398a8eb409fcae63a306485ae...,0000000000000a11127c06e40b008b4e9e2dc69b8335d4...,19812408076723b1b74138d84b0627ef09a1b9e3a7cc2f...,1333227325000,436883582,1031101468,1,69861,None,[{'transaction_id': '73822227ba5b23a8a9662fd06...,1


テーブルを取得

In [79]:
%%time
bq_assistant.list_tables()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 175 ms


['blocks', 'transactions']

blocksのカラム一覧を取得

In [18]:
%%time
bq_assistant.table_schema("blocks")

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 223 ms


,name,type,mode,description
0,block_id,STRING,NULLABLE,None
1,previous_block,STRING,NULLABLE,None
2,merkle_root,STRING,NULLABLE,None
3,timestamp,INTEGER,NULLABLE,None
4,difficultyTarget,INTEGER,NULLABLE,None
5,nonce,INTEGER,NULLABLE,None
6,version,INTEGER,NULLABLE,None
7,work_terahash,INTEGER,NULLABLE,None
8,work_error,STRING,NULLABLE,None
9,transactions,RECORD,REPEATED,None


transactionsのカラムを取得

In [19]:
%%time
bq_assistant.table_schema("transactions")

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 170 ms


,name,type,mode,description
0,timestamp,INTEGER,NULLABLE,None
1,transaction_id,STRING,NULLABLE,None
2,inputs,RECORD,REPEATED,None
3,inputs.input_script_bytes,BYTES,NULLABLE,None
4,inputs.input_script_string,STRING,NULLABLE,None
5,inputs.input_script_string_error,STRING,NULLABLE,None
6,inputs.input_sequence_number,INTEGER,NULLABLE,None
7,inputs.input_pubkey_base58,STRING,NULLABLE,None
8,inputs.input_pubkey_base58_error,STRING,NULLABLE,None
9,outputs,RECORD,REPEATED,None


In [167]:
QUERY = "SELECT * FROM `bigquery-public-data.bitcoin_blockchain.transactions` LIMIT 10"
df = bq_assistant.query_to_pandas(QUERY)

In [168]:
df

,timestamp,transaction_id,inputs,outputs,block_id,previous_block,merkle_root,nonce,version,work_terahash,work_error
0,1373660094000,3fa43eed0b4d92652c916c7cc4a7eee85486d4b58aa2a5...,[{'input_script_bytes': b'H0E\x02 \x1dO\xf0\xa...,"[{'output_satoshis': 34190000, 'output_script_...",00000000000000041a66e3c4847b495cd5ea7f73107504...,000000000000007dfc7749a8502412af88480d382443e2...,b1365e7401fac1be3c60ff12507f3deef20f6ef9b96b05...,4152738190,2,1123704,None
1,1413236055000,30e91a7e14ca7238313621bd93f5ceecf11942ae3f38ff...,[{'input_script_bytes': b'I0F\x02!\x00\xe0\x9c...,"[{'output_satoshis': 726457000, 'output_script...",00000000000000001ad5f971472868eee7fae7a31a7412...,000000000000000011964d8493d56e67082587d19e1231...,60cffc5d1e5a2fee55dcdaa7c4c42630000116d175e5e2...,2082302581,2,1503368095,None
2,1407155728000,e6ed95f395762d8cbc982cea93febbd985c91223c0901f...,[{'input_script_bytes': b'H0E\x02!\x00\x91a][L...,"[{'output_satoshis': 12776441, 'output_script_...",0000000000000000252b5ffbe95c5924ba9c3291c65477...,00000000000000001d21464dd4eadf9b3a20b82c737b3c...,a7d7b58f5e3097920278e4deec262176e18c2800056a55...,107858511,2,804736316,None
3,1365584430000,3fac4874a01d040a5be18f9503c3fdc3e58d0ba0d5d1ee...,"[{'input_script_bytes': b'G0D\x02 H""W5\xa6\x92...","[{'output_satoshis': 337855333, 'output_script...",00000000000001808afd19301d6e6a42ec0a17fb9edb60...,0000000000000082a1210dced8f8bbd13fde0ca8781b9d...,11dac12fda1fe487a195088526347f558318c076253a26...,2495270816,2,329557,None
4,1459076640000,376c6f3455c708109559e06be06d7fb94a38e4f8a63a23...,[{'input_script_bytes': b'\x00G0D\x02 \x12S\xf...,"[{'output_satoshis': 1042101, 'output_script_b...",00000000000000000620f0568e690305c791647006eb4e...,000000000000000000450eecdc5e5f64aaeb4a73d72028...,dbe910ff7872e96ccd3280e708402be8c57b82b7236a94...,3306265622,4,7108143405,None
5,1408858216000,978f907a9735d97d7d5dcbbc9ce83bc632e8a7ec7e04b5...,"[{'input_script_bytes': b'G0D\x02 \x1d\xdb""\x9...","[{'output_satoshis': 5899550, 'output_script_b...",0000000000000000255509f0b902c67c1c92867d6e32cf...,00000000000000000779b66b1a9bca420f29c59dca8aca...,458c329217e68e0572cef377a0866ef470bce90fac641c...,2178532378,2,1024136407,None
6,1343351178000,08f466620d06190ddbd808c71a4cb7d42e1606c11df572...,[{'input_script_bytes': b'I0F\x02!\x00\xe6\t@\...,"[{'output_satoshis': 3962696, 'output_script_b...",00000000000001a511182bf031724a2cf3d24907aece29...,00000000000001f69fb55c8e1ee81708ab0c4d392b6b7a...,55309d653a691e82b5302d2b0392b151c9eabd4e906c83...,639951838,1,80162,None
7,1430820198000,fd4b1a3d0c48d9394b336b37fa3126a1ce82569f3b9dd4...,[{'input_script_bytes': b'H0E\x02!\x00\xb00\xe...,"[{'output_satoshis': 200000, 'output_script_by...",00000000000000000e55d2b84f06836a0a132663ab6af1...,000000000000000008641293a1de7cbde40b24fc29b5cb...,c1fbdf8ad79f4c22501e0bd2458c1880e667c7321d4839...,2589450720,2,2046299587,None
8,1442521811000,c63853fb7f03fd9638e1c56bb82feb85dd5ca46517d511...,[{'input_script_bytes': b'H0E\x02!\x00\xab\x03...,"[{'output_satoshis': 21452782, 'output_script_...",00000000000000000fa13ca8747dfb4cb1f6959ac30ba4...,000000000000000008aec102f67979e6c7e14b9690fc85...,8999961445a0dcb9a34bd0330a2f7f7232798998cc1ab0...,1757085618,3,2548472817,None
9,1439239359000,491eb0ccab73a2b4f51e392c31c5ae1c55b5dd10413401...,[{'input_script_bytes': b'G0D\x02 2\xe9\xa7\xd...,"[{'output_satoshis': 52397171, 'output_script_...",00000000000000000ff6cbd5117be60ddb6fe207a29fce...,00000000000000000049aabe8c436e6b31883a7ec07912...,4d3005aa510e875a1ff56587bf609368ff9dc577c5422d...,2173297297,3,2263475534,None


In [169]:
df.outputs

0    [{'output_satoshis': 34190000, 'output_script_...
1    [{'output_satoshis': 726457000, 'output_script...
2    [{'output_satoshis': 12776441, 'output_script_...
3    [{'output_satoshis': 337855333, 'output_script...
4    [{'output_satoshis': 1042101, 'output_script_b...
5    [{'output_satoshis': 5899550, 'output_script_b...
6    [{'output_satoshis': 3962696, 'output_script_b...
7    [{'output_satoshis': 200000, 'output_script_by...
8    [{'output_satoshis': 21452782, 'output_script_...
9    [{'output_satoshis': 52397171, 'output_script_...
Name: outputs, dtype: object

送金額が一番高いユーザーを探す

In [160]:
q = """
select
    output.output_pubkey_base58 as reciver
    , sum(output.output_satoshis) as satoshi
from
    `bigquery-public-data.bitcoin_blockchain.transactions`
    JOIN UNNEST (outputs) AS output
group by reciver
"""
bq_assistant.estimate_query_size(q)

q = """
SELECT
    o.output_pubkey_base58,
    sum(o.output_satoshis) as output_sum 
from 
    `bigquery-public-data.bitcoin_blockchain.transactions`JOIN
    UNNEST(outputs) as o 
    where o.output_pubkey_base58 not in (select i.input_pubkey_base58
    from UNNEST(inputs) as i)
    group by o.output_pubkey_base58 order by output_sum desc limit 1000
"""
print (str(round((bq_assistant.estimate_query_size(q)),2))+str(" GB"))


62.03 GB


In [192]:
tran_data=bq_assistant.query_to_pandas(q)

In [193]:
tran_data.head()

,output_pubkey_base58,output_sum
0,1LNWw6yCxkUmkhArb2Nf2MPw6vG7u5WG7q,570640680858117
1,12cgpFdJViXbwHbhrA3TuW1EGnL25Zqc3P,444380792865258
2,3KgtbGgaX2ngstNpvyv7LwpHSweVeqGbpM,411801053119073
3,1NDyJtNTjmwk5xPNhjgAMu4HDHigtobu1s,323293346893794
4,13p5iQkqBEVgKmPeJqEL2LBRS44PjX1dZL,316971752467666


In [194]:
tran_data["now_bit"] = round(tran_data["output_sum"] * 0.00000001)

In [200]:
tran_data["now_money"]  = tran_data["now_bit"] * 600000 

In [201]:
tran_data.now_money[0]

3423844200000.0

In [202]:
tran_data.now_bit

0      5706407.0
1      4443808.0
2      4118011.0
3      3232933.0
4      3169718.0
5      2985433.0
6      2443411.0
7      2131017.0
8      2111772.0
9      2032974.0
10     1946266.0
11     1921923.0
12     1920989.0
13     1899811.0
14     1870447.0
15     1837108.0
16     1812586.0
17     1809059.0
18     1790014.0
19     1669261.0
20     1653099.0
21     1525838.0
22     1297715.0
23     1027463.0
24     1025064.0
25     1020227.0
26      989756.0
27      938648.0
28      934959.0
29      863023.0
         ...    
970      61742.0
971      61707.0
972      61626.0
973      61598.0
974      61517.0
975      61469.0
976      61389.0
977      61384.0
978      61236.0
979      61107.0
980      61107.0
981      61027.0
982      61008.0
983      61000.0
984      61000.0
985      60897.0
986      60863.0
987      60816.0
988      60710.0
989      60665.0
990      60643.0
991      60529.0
992      60449.0
993      60424.0
994      60377.0
995      60369.0
996      60253.0
997      60123

total６兆円ほどの額が動いてる計算になります。よくわかりません。